[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/witchapong/build-ai-based-applications/blob/main/llm/2_news_summarization_and_information_extraction.ipynb)

# News Summarization and Information Extraction

In this notebook, we'll learn how to build LLMs application for summarize and also extract useful key informations from lengthy news or articles.

The content of this notebook is mainly modified from content from [KBTG M.A.D.Bootcamp](https://kbtgkampus.tech/).

In [ ]:
!pip install -q langchain==0.2.2
!pip install -q langchain-openai==0.1.8

## Generate OpenAI key

Generate key from: https://platform.openai.com/api-keys

In [ ]:
import os

# TODO: put your generated openai key here
os.environ["OPENAI_API_KEY"] = "YOUR-SECRET-API-KEY"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [ ]:
from langchain.globals import set_debug
set_debug(False)

In [ ]:
news_text = """
Inflation, interest rates and tariffs mean 2025 is shaping up to be an intriguing year for the global economy. One in which growth is expected to remain at a "stable yet underwhelming" 3.2%, according to the International Monetary Fund. So what might that mean for all of us?
Exactly a week before Christmas there was a welcome gift for millions of American borrowers - a third interest rate cut in a row.
However, stock markets fell sharply because the world's most powerful central banker, US Federal Reserve chair Jerome Powell, made clear they shouldn't expect as many further cuts in 2025 as they might have hoped for, as the battle against inflation continues.
"From here, it's a new phase, and we're going to be cautious about further cuts," he said.
In recent years, the Covid pandemic and the war in Ukraine have led to sharp price rises around the world, and although prices are still increasing the pace has slowed markedly.
Despite that, November saw inflation push up in the US, eurozone and UK to to 2.7%, 2.2% and 2.6% respectively. It highlights the difficulties many central banks face in the so-called "last mile" of their battle against inflation. Their target is 2%, and it might be easier to achieve if economies are growing.
However, the biggest difficulty for global growth "is uncertainty, and the uncertainty is coming from what may come out of the US under Trump 2.0", says Luis Oganes, who is head of global macro research at investment bank JP Morgan.

Since Donald Trump won November's election he's continued to threaten new tariffs against key US trading partners, China, Canada and Mexico.
"The US is going into a more isolationist policy stance, raising tariffs, trying to provide more effective protection to US manufacturing," says Mr Oganes.
"And even though that is going to support US growth, at least in the short term, certainly it's going to hurt many countries that rely on trade with the US."
New tariffs "could be particularly devastating" for Mexico and Canada, but also be "harmful" to the US, according to Maurice Obstfeld, a former chief economist at the International Monetary Fund, and a previous economic advisor to President Obama.
He cites car manufacturing as an example of an industry that "depends on a supply chain that is spread across the three countries. If you disrupt that supply chain, you have massive disruptions in the auto market".
That has the potential to push up prices, reduce demand for products, and hurt company profits, which could in turn drag down investment levels, he explains.
Mr Obstfeld, who is now with the Peterson Institute for International Economics, adds: "Introducing these types of tariffs into a world that is heavily dependent on trade could be harmful to growth, could throw the world into recession."
The tariffs threats have also played a role in forcing the resignation of Canada's Prime Minister Justin Trudeau.

Even though the majority of what the US and China sell each other is already subject to tariffs from Donald Trump's first term in office, the threat of new tariffs is a key challenge for the world's second-biggest economy in the year ahead.
In his new year address President Xi Jinping acknowledged the "challenges of uncertainties in the external environment", but said the economy was on "an upward trajectory".
Exports of cheap goods from its factories are crucial to China's economy. A drop off in demand because tariffs push prices up would compound the many domestic challenges, including weak consumer spending and business investment, that the government is trying to tackle.
Those efforts are helping, according to the World Bank, which at the end of December increased its forecast for China's growth from 4.1% to 4.5% in 2025.
Beijing has yet to set a growth target for 2025, but thinks it's on course for 5% last year.
"Addressing challenges in the property sector, strengthening social safety nets, and improving local government finances will be essential to unlocking a sustained recovery," according to the World Bank's country director for China, Mara Warwick.
Those domestic struggles mean the Chinese government is "more welcoming" of foreign investment, according to Michael Hart, who is president of the American Chamber of Commerce in China.
Tensions between the US and China, and tariffs have grown under the Biden presidency, meaning some companies have looked to move production elsewhere.
However, Mr Hart points out that "it took 30 to 40 years for China to emerge as such a strong supplier manufacturer", and whilst "companies have tried to mitigate some of those risks... no one's prepared now to completely replace China."

One industry that is likely to continue to be at the heart of global trade battles is electric vehicles. More than 10 million were made in China last year, and that dominance led the US, Canada and European Union (EU) to impose tariffs on them.
Beijing says they're unfair, and is challenging them at the World Trade Organization.
However, it's the prospect of Donald Trump imposing tariffs that is concerning the EU.
"Restrictions on trade, protectionist measures, are not conducive to growth, and ultimately have an impact on inflation that is largely uncertain," the president of the European Central Bank, Christine Lagarde, said last month. "[But] in the short term, it's probably net inflationary."
Germany and France are the traditional engines of Europe's economic growth. But their poor performance amid political instability over the past year means that, despite a recent uptick in growth, the eurozone risks losing momentum in the year ahead.
That is, unless consumers spend more and businesses increase their investments.
In the UK higher prices could also come as a result of tax and wage increases, according to one survey.
One barrier to cutting eurozone interest rates is that domestic inflation, which focuses on the prices of items that are less prone to influence from external factors, remains at 4.2%. That's more than double the overall inflation target of 2%, and strong wage pressure has been a barrier getting it down further.
It's been similar in the US according to Sander van 't Noordende, the chief executive of Randstad, the world's biggest recruitment firm.
"In the US, for instance, [wage inflation] is still going to be around 4% in 2024. In some Western European countries, it's even higher than that.
"I think there's two factors there. There's the talent scarcity, but there's also, of course, the inflation and people demanding to get more for the work they do."
Mr van 't Noordende adds that many companies are passing those extra costs on to their customers, which is adding upward pressure to general inflation.
A slowdown in the global jobs market reflects a lack of "dynamism" from companies and economic growth is key to reversing that, he says.
"If the economy is doing well, businesses are growing, they start hiring. People see interesting opportunities, and you just start seeing people moving around".

One person starting a new role in 2025 is Donald Trump, and a raft of economic plans including tax cuts and deregulation could help the US economy to continue to thrive.
Whilst much won't be revealed before he's back in the White House on 20 January, "everything points to continued US exceptionalism at the expense of the rest of the world," says JP Morgan's Mr Oganes.
He's hopeful that inflation and interest rates can continue to come down around the world, but warns that "a lot of it will depend on what are the policies that get deployed, particularly from the US."
"""

## Building Information Extraction AI

In [ ]:
# define langchain model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.05)

# write system prompts
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
"""
You will be provided with lengthy news or article, and your task is summarizing and extracting key information from it.
You will output the following JSON object with these keys:
- summary: string, executive summary of given news or article hilighting i) the what the key information is and ii) why it is important in no more than 3 sentences.
- key_information: list of strings, no more than 5 most relevant main points supporting the summary.
- related_entities: list of strings, all related entities mentioned in the news or article.
Answer in the form of a JSON object.
"""
),
        ("user", "{news}")
    ]
)

parser = JsonOutputParser()

chain = prompt_template | model | parser

In [ ]:
chain.invoke({"news": news_text})

{'summary': 'The global economy in 2025 is expected to see stable yet underwhelming growth of 3.2%, with challenges such as inflation, interest rates, and tariffs impacting various countries. The uncertainty surrounding policies, particularly from the US under Trump 2.0, is a major concern for global growth and trade relationships, with potential repercussions on industries like car manufacturing and electric vehicles.',
 'key_information': ['Global growth in 2025 is projected to be 3.2%, with inflation rates rising in the US, eurozone, and UK.',
  'Tariffs imposed by the US under Trump 2.0 could harm countries like Mexico, Canada, and China, impacting industries and trade relationships.',
  "China's economy faces challenges from tariffs, weak consumer spending, and business investment, despite efforts to address domestic struggles.",
  'Electric vehicles are at the center of global trade battles, with tariffs imposed by the US, Canada, and EU on Chinese-made vehicles.',
  'The eurozon

## Add support for multi languages!!

In [ ]:
# define langchain model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.01)

# write system prompts
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
"""
You will be provided with lengthy news or article in any language. Your task is detecting an input language name, and summarizing and extracting key information from it in standard {language} language.
You will output the following JSON object with these keys:
- summary: string, executive summary of given news or article hilighting i) the what the key information is and ii) why it is important in no more than 3 sentences.
- key_information: list of strings, no more than 5 most relevant main points supporting the summary.
- related_entities: list of strings, all related entities mentioned in the news or article.
Answer in the form of a JSON object.
- input_language: string, name of the detected input language.
"""
),
        ("user", "{news}")
    ]
)

parser = JsonOutputParser()

chain = prompt_template | model | parser

In [ ]:
chain.invoke({"news": news_text, "language": "Thai"})

{'summary': 'ปี 2025 จะเป็นปีที่น่าสนใจสำหรับเศรษฐกิจโลก เนื่องจากการเจริญเติบโตที่คาดว่าจะคงอยู่ที่ระดับ 3.2% ตามรายงานจากสำนักงานการเงินนานาชาติ แม้ว่าการต่อสู้กับอินฟเลชั่นยังคงต่อเนื่อง แต่ความไม่แน่นอนที่มาจากนโยบายของสหรัฐฯ ในยุค Trump 2.0 อาจส่งผลกระทบต่อการเจริญเติบโตของโลก',
 'key_information': ['การเจริญเติบโตของเศรษฐกิจโลกคาดว่าจะคงอยู่ที่ระดับ 3.2%',
  'การต่อสู้กับอินฟเลชั่นยังคงเป็นปัญหาสำคัญสำหรับธนาคารกลางหลายแห่ง',
  'การเสนอภาษีซึ่งอาจส่งผลกระทบต่อการเจริญเติบโตของโลก',
  'การเจริญเติบโตของจีนที่เพิ่มขึ้นจาก 4.1% เป็น 4.5% ในปี 2025',
  'ความไม่แน่นอนที่มาจากนโยบายของสหรัฐฯ ในยุค Trump 2.0 อาจส่งผลกระทบต่อการเจริญเติบโตของโลก'],
 'related_entities': ['International Monetary Fund',
  'US Federal Reserve',
  'Jerome Powell',
  'Luis Oganes',
  'JP Morgan',
  'Donald Trump',
  'China',
  'Canada',
  'Mexico',
  'Maurice Obstfeld',
  'International Monetary Fund',
  'President Obama',
  'Justin Trudeau',
  'President Xi Jinping',
  'World Bank',
  'Mara Warwick',
  'Micha

In [ ]:
chain.invoke({"news": news_text, "language": "Spanish"})

{'summary': 'El año 2025 se perfila como un año intrigante para la economía global, con desafíos como la inflación, las tasas de interés y los aranceles. La incertidumbre proviene de las políticas aislacionistas y los nuevos aranceles de Estados Unidos, lo que podría afectar el crecimiento mundial y desencadenar una recesión. La lucha contra la inflación y las decisiones de la Reserva Federal de EE. UU. tendrán un impacto significativo en la economía mundial.',
 'key_information': ['La economía global se espera que crezca un 3.2% en 2025, según el Fondo Monetario Internacional.',
  'Los aranceles amenazados por Donald Trump podrían ser perjudiciales para el crecimiento mundial y desencadenar una recesión.',
  'La incertidumbre en la economía global proviene de las políticas aislacionistas de EE. UU. y la lucha contra la inflación.',
  'China enfrenta desafíos debido a los aranceles y la disminución de la demanda de sus productos debido a los precios más altos.',
  'La Eurozona y el Rei